In [1]:
import pandas as pd
from scipy import stats
import json

#Load protein dictionaries
with open('shortened_protein_dict.json') as file:
    protein_dict = json.load(file)

with open('shortened_protein_translations.json') as file:
    protein_translations = json.load(file)
    
with open('protein_translations.json') as file:
    full_protein_translations = json.load(file)
    
with open('protein_dict.json') as file:
    full_protein_dict = json.load(file)

In [2]:
phages_df = pd.read_csv('CombinedData.csv')
sequenced_df = pd.read_csv('SequencedPhages.csv')
sequenced_df = sequenced_df[['Phage Name', 'Host']]

phages_df = pd.merge(phages_df, sequenced_df, on='Phage Name')

In [43]:
#phages_df.to_csv('CombinedData.csv', index=False)

In [3]:
results_df = pd.read_csv('protein_results.csv')

In [6]:
#Only include proteins with a certain number of phages
shortened_dict = {}

for key in protein_dict.keys():
    if len(protein_dict[key]) > 19:
        shortened_dict[key] = protein_dict[key]


protein_dict = shortened_dict


In [40]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#Fuzzy string comparison to match proteins
def fuzzy_protein_match(protein):
    phages_list = []
    string = protein_translations[protein]
    for key in full_protein_translations:
        if fuzz.ratio(string, full_protein_translations[key]) > 95:
            for phage in full_protein_dict[key]:
                if phage not in phages_list:
                    #Append phage to protein_dict if fuzzy match is found
                    phages_list.append(phage)
    return phages_list

In [39]:
#protein_name = 'Protein 334'

#print(len(protein_dict[protein_name]))
#fuzzy_protein_match(protein_name)
#print(len(protein_dict[protein_name]))
#print(len(protein_dict['Protein 2575']))
#protein_dict1 = fuzzy_protein_match('Protein 2575')
#print(len(protein_dict1))

In [95]:
#Chi square test for homogeneity
def chi2_test(key, factor, bins):
    qcut_col = f'{factor}_qcut'
    df = host_df[['Phage Name', factor]]
    
    #Create labels list of length {quantiles}
    labels = []
    for i in range(bins):
        labels.append(i)
        
    #Separate into equal quantiles using qcut
    df[qcut_col] = pd.qcut(host_df[factor], q=bins,
                                labels=labels)
    
    #Create two df, one with protein and one without
    has_protein = df[df['Phage Name'].isin(protein_dict[key])]
    no_protein = df[df['Phage Name'].isin(protein_dict[key]) == False]
    
    #Initialize empty lists for counts for each bin
    yes_counts = []
    no_counts = []
    
    #Append counts to contigency table row lists
    for label in labels:
        yes_counts.append(len(has_protein[has_protein[qcut_col] == label]))
        no_counts.append(len(no_protein[no_protein[qcut_col] == label]))
 
    #Contingency table
    data = [yes_counts, no_counts]
    
    return data


In [96]:
#Second chi square test for homogeneity using fuzzy protein match
def chi2_test_fuzzy(key, factor, bins):
    qcut_col = f'{factor}_qcut'
    df = host_df[['Phage Name', factor]]
    
    #Create labels list of length {quantiles}
    labels = []
    for i in range(bins):
        labels.append(i)
        
    #Separate into equal quantiles using qcut
    df[qcut_col] = pd.qcut(host_df[factor], q=bins,
                                labels=labels)
    
    phages_list = fuzzy_protein_match(key)
    
    #Create two df, one with protein and one without
    has_protein = df[df['Phage Name'].isin(phages_list)]
    no_protein = df[df['Phage Name'].isin(phages_list) == False]
    
    #Initialize empty lists for counts for each bin
    yes_counts = []
    no_counts = []
    
    #Append counts to contigency table row lists
    for label in labels:
        yes_counts.append(len(has_protein[has_protein[qcut_col] == label]))
        no_counts.append(len(no_protein[no_protein[qcut_col] == label]))
 
    #Contingency table
    data = [yes_counts, no_counts]
    
    return data


In [80]:
###spurious_results_df = pd.DataFrame({'Protein': [], 'P value': [], 'Factor': [], 'Host': [], 'Has protein': [], 'No protein': [], 'Translation': []})

In [42]:
#Limit to one host
host = 'Microbacterium foliorum NRRL B-24224'
host_df = phages_df[phages_df['Host'] == host]

#host_df['Zipcode'] = pd.to_numeric(host_df['Zipcode'], errors='coerce')

In [31]:
#quartiles_df = pd.DataFrame({'Factor': [], 'Min': [], 'Q1': [], 'Med': [],
                         #'Q3': [], 'Max': [], 'Host': []})

In [41]:
#host_df['Zipcode'] = pd.to_numeric(host_df['Zipcode'], errors='coerce')
#host_df = host_df[host_df['Zipcode'] >= 10000]

for factor in factors:
    quartiles = pd.qcut(host_df[factor], q=4, retbins=True)
    print(quartiles[1])
    #temp_df = pd.DataFrame({'Factor': [factor], 'Min': [quartiles[1][0]], 'Q1': [quartiles[1][1]], 'Med': [quartiles[1][2]],
                         #'Q3': [quartiles[1][3]], 'Max': [quartiles[1][4]], 'Host': [host]})
    #quartiles_df= quartiles_df.append(temp_df, ignore_index='False')

[10035. 15260. 23061. 43950. 99258.]


C:\Users\Chenchen\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
quartiles_df.to_csv('Factor Quartiles.csv', index=False)

In [115]:
def test_proteins(factor):
    yes1 = 0
    yes2= 0
    #Iterate through proteins 
    for key in protein_dict.keys():
        #Number of bins
        bins = 4
        #Significance level (alpha)
        sig_lvl1 = 0.001
        sig_lvl2 = 0.001

        data = chi2_test(key, factor, bins)
        #Check that all counts are >= 5
        if all(i >= 5 for i in data[0]):
            p_value = stats.chi2_contingency(data)[1]
            #If p value is significant using exact matches
            if p_value < sig_lvl1:
                yes1+= 1
                #Run the test again, using fuzzy string match to expand list of phages with that protein
                data_fuzzy = chi2_test_fuzzy(key, factor, quantiles)
                p_value_fuzzy = stats.chi2_contingency(data_fuzzy)[1]
                #If fuzzy p value is significant
                if p_value_fuzzy < sig_lvl2:
                    yes2+= 1
                    temp_df = pd.DataFrame({'Protein': [key], 'P value': [p_value_fuzzy], 'Factor': [factor], 'Host': [host],
                                                    'Has protein': [data_fuzzy[0]], 'No protein': [data_fuzzy[1]], 'Translation': [protein_translations[key]]})

                    results_df = results_df.append(temp_df, ignore_index='False')


                print(factor)
                print(key)
                print(f"P-value: {p_value_fuzzy}")
                print(f"Protein translation: {protein_translations[key]}")
                print(f"Has protein: {data_fuzzy[0]}")
                print(f"Does not have protein: {data_fuzzy[1]}")
                print('\n\n')


    print(yes1
    print(yes2

In [16]:
factors = ['AQI', 'GHI', 'pH', 'Population Density', 'Dew Point', 'Albedo']
for factor in factors:
    test_proteins(factor)

In [ ]:
#Only using exact string comparison, no fuzzy compare

#Limit to one host
host = 'Microbacterium foliorum NRRL B-24224
host_df = phages_df[phages_df['Host'] == host]
#host_df['Zipcode'] = pd.to_numeric(host_df['Zipcode'], errors='coerce')


#print(str(len(host_df)) + ' phages')
yes = 0

#Iterate through proteins 
for key in protein_dict.keys():
    #Factor of interest
    factor = 'Longitude'
    #Number of bins
    bins = 4
    #Significance level (alpha)
    sig_lvl = 0.001
    
    data = chi2_test(key, factor, bins)
    #Check that all counts are >= 5
    if all(i >= 5 for i in data[0]):
        p_value = stats.chi2_contingency(data)[1]
        #If p value is significant using exact matches
        if p_value < sig_lvl:
            temp_df = pd.DataFrame({'Protein': [key], 'P value': [p_value], 'Factor': [factor], 'Host': [host],
                                                'Has protein': [data[0]], 'No protein': [data[1]], 'Translation': [protein_translations[key]]})

            ###results_df = results_df.append(temp_df, ignore_index='False')


            print(factor)
            print(key)
            print(f"P-value: {p_value}")
            print(f"Protein translation: {protein_translations[key]}")
            print(f"Has protein: {data[0]}")
            print(f"Does not have protein: {data[1]}")
            print('\n\n')
            yes += 1
   
print(yes)


In [90]:
phages_df['Host'].value_counts()

Mycobacterium smegmatis mc¬≤155               1560
Gordonia terrae 3612                           331
Microbacterium foliorum NRRL B-24224           261
Arthrobacter sp.  ATCC 21022                   203
Streptomyces griseus ATCC 10137                 88
Arthrobacter globiformis B-2979                 76
Gordonia rubripertincta NRRL B-16540            36
Rhodococcus erythropolis RIA 643                34
Microbacterium paraoxydans NWU1                 27
Streptomyces lividans JI 1326                   25
Propionibacterium acnes ATCC 6919               25
Gordonia terrae CAG3                            24
Streptomyces xanthochromogenes NRRL B-5410      18
Streptomyces viridochromogenes DSM40736         15
Streptomyces venezuelae ATCC 10712              14
Streptomyces scabiei RL-34                      14
Gordonia terrae NRRL B-16283                    13
Microbacterium paraoxydans NRRL B-14843         13
Streptomyces griseofuscus ATCC 23916            13
Corynebacterium xerosis ATCC 37

In [116]:
#results_df.to_csv('protein_results_fuzzy.csv', index=False)

In [89]:
#spurious_results_df.to_csv('Spurious Results.csv', index=False)

In [ ]:
#T-test between phages with protein and without protein

phages_df = phages_df[phages_df['Host'] == 'Mycobacterium smegmatis mc¬≤155']

for key in protein_dict.keys():
    has_protein = phages_df[phages_df['Phage Name'].isin(protein_dict[key])]
    no_protein = phages_df[phages_df['Phage Name'].isin(protein_dict[key]) == False]
    factor = 'AQI'
    has_protein = has_protein.dropna(subset=[factor])
    no_protein = no_protein.dropna(subset=[factor])
    p_value = stats.ttest_ind(has_protein[factor], no_protein[factor])[1]
    if p_value < 0.001:
        print(key)
        print(p_value)
        print(f"Has protein: {has_protein[factor].mean()}")
        print(f"Does not have protein: {no_protein[factor].mean()}")
        print(f"Protein translation: {protein_translations[key]}")
        print()
    